In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from keras.constraints import maxnorm
from keras.preprocessing.image import ImageDataGenerator

In [2]:
DATA_DIR = "/content/drive/MyDrive/malware_imagenes/malimg_paper_dataset_imgs"

In [3]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### PREPARACION

#### Carga de datos

In [4]:
path_images = ImageDataGenerator().flow_from_directory(directory=DATA_DIR, target_size=(128,128), batch_size=10000)

Found 9339 images belonging to 25 classes.


In [5]:
images, marks = next(path_images)

In [6]:
indices_path = path_images.class_indices

In [7]:
type_images = indices_path.keys()

In [8]:
add = sum(marks)
marks_qt = marks.shape[0]

In [9]:
quantity = ((add) / marks_qt)

In [10]:
quantity = quantity * 100
quantity

array([ 1.3063496 ,  1.2421031 , 31.577257  , 17.036085  ,  2.1201413 ,
        1.1350251 ,  1.5633365 ,  2.141557  ,  1.895278  ,  1.7346611 ,
        4.079666  ,  4.615055  ,  2.2807581 ,  1.9702322 ,  1.3170575 ,
        1.7025378 ,  1.4562588 ,  1.5205053 ,  1.6918299 ,  0.85662276,
        1.3705964 ,  1.4134276 ,  4.3687763 ,  1.038655  ,  8.566228  ],
      dtype=float32)

#### Preparacion de datos

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(images/255.,marks, test_size=0.3)

In [13]:
X_train.shape

(6537, 128, 128, 3)

In [14]:
X_test.shape

(2802, 128, 128, 3)

In [15]:
y_train.shape

(6537, 25)

In [16]:
y_test.shape

(2802, 25)

### MODELO BASE

In [48]:
base_model = keras.Sequential()
base_model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
base_model.add(keras.layers.MaxPooling2D(2))
base_model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#base_model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
base_model.add(keras.layers.Dense(64))
base_model.add(keras.layers.Flatten())
base_model.add(keras.layers.Dense(25, activation='softmax'))

In [49]:
base_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 dense_6 (Dense)             (None, 62, 62, 64)        4160      
                                                                 
 flatten_3 (Flatten)         (None, 246016)            0         
                                                                 
 dense_7 (Dense)             (None, 25)                6150425   
                                                      

#### Compilado

In [50]:
base_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#### Clasificador

In [51]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

In [52]:
tf.compat.v1.disable_eager_execution()

In [53]:
classifier = KerasClassifier(
    model=base_model,
    clip_values=(0, 1),
    use_logits=False
)

#### Entrenamiento

In [55]:
classifier.fit(X_train, y_train, batch_size=64, nb_epochs=2)

Train on 6537 samples
Epoch 1/2
6537/6537 [==============================] - 208s 32ms/sample - loss: 1.4835 - accuracy: 0.7647
Epoch 2/2
6537/6537 [==============================] - 193s 30ms/sample - loss: 0.0958 - accuracy: 0.9726


#### Predicciones

In [56]:
predictions = np.argmax(classifier.predict(X_test), axis=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [57]:
correct_predicted = np.sum(predictions == np.argmax(y_test, axis=1))
acc = correct_predicted / len(y_test)

print("Accuracy para predicciones de modelos base: {}%".format(acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(correct_predicted))

Accuracy para predicciones de modelos base: 95.53890078515346%
Cantidad de valores clasificados correctamente: 2677


#### Generacion de observaciones

In [58]:
attacker = FastGradientMethod(estimator=classifier, eps=50)

In [59]:
x_test_adv = attacker.generate(x=X_test)

#### Prediccion y resultados con observaciones falsas

In [60]:
adv_predictions = np.argmax(classifier.predict(x_test_adv), axis=1)

In [61]:
adv_correct_predicted = np.sum(adv_predictions == np.argmax(y_test, axis=1))
adv_acc = adv_correct_predicted / len(y_test)

print("Accuracy para modelos con ataque: {}%".format(adv_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(adv_correct_predicted))

Accuracy para modelos con ataque: 2.426837972876517%
Cantidad de valores clasificados correctamente: 68


### MODELO MEJORADO

#### Muestras adversarias

In [30]:
train_attacker = FastGradientMethod(estimator=classifier, eps=100)

In [31]:
x_train_fgm = attacker.generate(x=X_train)

In [32]:
X_train = np.append(X_train, x_train_fgm, axis=0)
y_train = np.append(y_train, y_train, axis=0)

#### Entrenamiento

In [ ]:
classifier.fit(X_train, y_train, batch_size=64, nb_epochs=3)

#### Predicciones y evaluacion

In [ ]:
rob_predictions = np.argmax(classifier.predict(x_test_adv), axis=1)

In [ ]:
rob_correct_predicted = np.sum(rob_predictions == np.argmax(y_test, axis=1))
rob_acc = rob_correct_predicted / len(y_test)

print("Accuracy de modelo mejorado con ataque: {}%".format(rob_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(len(rob_correct_predicted)))

### Trainer de ART

#### Modelo

In [62]:
defense_model = keras.Sequential()
defense_model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
defense_model.add(keras.layers.MaxPooling2D(2))
defense_model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
defense_model.add(keras.layers.Dense(64))
defense_model.add(keras.layers.Flatten())
defense_model.add(keras.layers.Dense(25, activation='softmax'))

In [63]:
defense_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 dense_8 (Dense)             (None, 62, 62, 64)        4160      
                                                                 
 flatten_4 (Flatten)         (None, 246016)            0         
                                                                 
 dense_9 (Dense)             (None, 25)                6150425   
                                                      

In [64]:
defense_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#### Classifier

In [65]:
dclassifier = KerasClassifier(
    model=defense_model,
    clip_values=(0, 1),
    use_logits=False
)

#### Entrenamiento

In [66]:
dclassifier.fit(X_train, y_train, batch_size=64, nb_epochs=2)

Train on 6537 samples
Epoch 1/2
6537/6537 [==============================] - 192s 29ms/sample - loss: 1.3408 - accuracy: 0.7785
Epoch 2/2
6537/6537 [==============================] - 188s 29ms/sample - loss: 0.0670 - accuracy: 0.9819


#### Predicciones

In [67]:
predictions = np.argmax(dclassifier.predict(X_test), axis=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [68]:
correct_predicted = np.sum(predictions == np.argmax(y_test, axis=1))
acc = correct_predicted / len(y_test)

print("TOTAL DE VALORES: {}".format(len(y_test)))
print("Accuracy de modelo mejorado con ataque: {}%".format(acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(correct_predicted))

TOTAL DE VALORES: 2802
Accuracy de modelo mejorado con ataque: 96.60956459671664%
Cantidad de valores clasificados correctamente: 2707


#### Entrenamiento adversario

In [69]:
dattacker = FastGradientMethod(estimator=dclassifier, eps=100)

In [70]:
x_test_adv = dattacker.generate(x=X_test)

In [71]:
from art.defences.trainer import AdversarialTrainer

In [80]:
defender = AdversarialTrainer(
    classifier=dclassifier,
    attacks=dattacker,
    ratio=0.5
)

In [81]:
defender.fit(x=X_train, y=y_train, nb_epochs=3)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/3 [00:00<?, ?it/s]

In [82]:
av_predictions = np.argmax(dclassifier.predict(x_test_adv), axis=1)

In [83]:
av_correct_predicted = np.sum(av_predictions == np.argmax(y_test, axis=1))
av_acc = av_correct_predicted / len(y_test)

print("TOTAL DE VALORES: {}".format(len(y_test)))
print("Accuracy de modelo mejorado con ataque: {}%".format(av_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(av_correct_predicted))

TOTAL DE VALORES: 2802
Accuracy de modelo mejorado con ataque: 85.36759457530336%
Cantidad de valores clasificados correctamente: 2392


#### Gráfica

In [47]:
N = len(y_test)

eps = [0.5, 5, 10, 30, 50, 100]

classifier_acc = []
defender_acc = []


for e in eps:
  attacker.set_params(**{'eps': e})
  dattacker.set_params(**{'eps': e})

  x_test_adv_1 = attacker.generate(x=X_test)
  x_test_adv_2 = dattacker.generate(x=X_test)

  predictions_1 = np.sum(np.argmax(classifier.predict(x_test_adv_1), axis=1) == np.argmax(y_test, axis=1))
  predictions_2 = np.sum(np.argmax(defender.predict(x_test_adv_2), axis=1) == np.argmax(y_test, axis=1))

  classifier_acc.append(predictions_1 / N)
  defender_acc.append(predictions_2 / N)

print(classifier_acc)
print(defender_acc)

[0.029621698786581014, 0.029621698786581014, 0.029621698786581014, 0.029621698786581014, 0.029621698786581014, 0.029621698786581014]
[0.9632405424696645, 0.9632405424696645, 0.9632405424696645, 0.9632405424696645, 0.9632405424696645, 0.9632405424696645]


Podemos observar que el modelo que ha pasado por el adversarial training es mucho más resiliente y se adapta mejor a las muestras falsas, de modo que logra clasificarlas correctamente.